In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from Suleima.core.preprocess import *
from mydataloader import * 
from utils import *
from SampleInfos import SampleInfo 
from Suleima.core.SamplePaths import SamplePath
#import pyradiomics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
#from totalsegmentator.python_api import totalsegmentator

import nibabel as nib
from nibabel.orientations import aff2axcodes
#from nibabel import Nifti1Image

#import dicom2nifti
#import dicom2nifti.convert_dicom

import SimpleITK as sitk
#import scipy.ndimage
#from skimage import measure
#import imageio


#from monai.data.meta_tensor import MetaTensor


import torch
import torch.nn as nn
import torch.optim as optim
import shap

from sklearn.model_selection import StratifiedShuffleSplit

import os
import pandas as pd
import numpy as np
import csv


import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap
from ipywidgets import interact


import re
import shutil


In [2]:
NORMAL='normal_cases'
TAKO='takotsubo_cases'

case_path=f"{NORMAL}/LAAB 06958326_90M"  #OG shape (512, 111, 512)
case_path=f"{NORMAL}/SS 10937555_61F"   #OG shape (512, 86, 512)

base_dicom_root = f"../Takotsubo-Syndrome/data/Inputs/{case_path}" 
base_input_root = f"data/Inputs/{case_path}"
base_output_root = f"data/Outputs/{case_path}"


In [ ]:
for patientID in os.listdir(dicom_root):

In [ ]:
def resample_CT(volume_np, 
                spacing, 
                new_spacing=[1.0, 1.0, 1.0],
                target_shape=(64, 64, 64), 
                original_affine=None):
    """
    Resamples a CT volume using linear interpolation.
    """
    sitk_img = sitk.GetImageFromArray(np.transpose(volume_np, (2, 1, 0)))
    sitk_img.SetSpacing([float(s) for s in spacing[::-1]])

    resampler = sitk.ResampleImageFilter()
    resampler.SetInterpolator(sitk.sitkLinear)
    
    original_size = np.array(sitk_img.GetSize(), dtype=np.int32)
    original_spacing = np.array(sitk_img.GetSpacing())
    new_size = np.round(original_size * (original_spacing / new_spacing)).astype(int).tolist()
    resampler.SetSize(new_size)
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetOutputDirection(sitk_img.GetDirection())
    resampler.SetOutputOrigin(sitk_img.GetOrigin())
    resampled = resampler.Execute(sitk_img)

    direction = np.array(resampled.GetDirection()).reshape(3, 3)
    spacing_arr = np.array(resampled.GetSpacing())

    origin = original_affine[:3, 3] if original_affine is not None else np.array(resampled.GetOrigin())
    affine = np.eye(4)
    affine[:3, :3] = direction * spacing_arr[:, None]
    affine[:3, 3] = origin

    resampled_np = np.transpose(sitk.GetArrayFromImage(resampled), (2, 1, 0))
    zoom_factors = [t / s for t, s in zip(target_shape, resampled_np.shape)]
    volume_final = scipy.ndimage.zoom(resampled_np, zoom=zoom_factors, order=1)

    return volume_final, affine, resampled


def resample_mask(volume_np, reference_image,
                  target_shape=(64, 64, 64), original_affine=None):
    """
    Resamples a binary or label mask using nearest-neighbour interpolation,
    aligned to the given reference image (typically a CT volume).
    """
    # Convert to SimpleITK image (z, y, x) format
    sitk_img = sitk.GetImageFromArray(np.transpose(volume_np, (2, 1, 0)))

    # Match mask to reference image geometry
    sitk_img.SetSpacing(reference_image.GetSpacing())  # optional but safe

    resampler = sitk.ResampleImageFilter()
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    resampler.SetReferenceImage(reference_image)

    resampled = resampler.Execute(sitk_img)

    # Recover affine from reference image and spacing
    direction = np.array(resampled.GetDirection()).reshape(3, 3)
    spacing_arr = np.array(resampled.GetSpacing())
    origin = original_affine[:3, 3] if original_affine is not None else np.array(resampled.GetOrigin())

    affine = np.eye(4)
    affine[:3, :3] = direction * spacing_arr[:, None]
    affine[:3, 3] = origin

    # Convert to NumPy in (x, y, z) order
    resampled_np = np.transpose(sitk.GetArrayFromImage(resampled), (2, 1, 0))

    # Resize to uniform shape
    zoom_factors = [t / s for t, s in zip(target_shape, resampled_np.shape)]
    volume_final = scipy.ndimage.zoom(resampled_np, zoom=zoom_factors, order=0)

    return volume_final, affine, resampled


def crop_trim_resample(case, Segment_root):
    # === Step 3 - Crop, trim, and resample heart ===

    OG_file = load_nifti(get_segments_paths(Segment_root))
    lv = OG_file["LV"]["data"]
    rv = OG_file["RV"]["data"]
    la = OG_file["LA"]["data"]
    ra = OG_file["RA"]["data"]
    myo = OG_file["MYO"]["data"]
    og_ct_data = OG_file["CT"]["data"]
    og_ct_voxel = OG_file["CT"]["voxel"]
    og_affine=OG_file["CT"]["affine"]
    # Combine masks to find heart bounding box
    binary_mask = ((lv + rv + la + ra + myo) > 0).astype(np.uint8)   # (sum of positive labels across all masks)  & turn the summed array into a binary mask
    x_min, y_min, z_min = np.array(np.where(binary_mask)).min(axis=1)
    x_max, y_max, z_max = np.array(np.where(binary_mask)).max(axis=1)
    
    # calculate crop boundaries (3D bounding box) 
    # identify min and max voxel coordinates where heart structures are present
    x0, x1 = max(x_min, 0), min(x_max, og_ct_data.shape[0])
    y0, y1 = max(y_min, 0), min(y_max, og_ct_data.shape[1])
    z0, z1 = max(z_min, 0), min(z_max, og_ct_data.shape[2])
    
    # crop volumes with bounding box coordinates
    ct_crop = og_ct_data[x0:x1, y0:y1, z0:z1]
    lv_crop = lv[x0:x1, y0:y1, z0:z1]
    rv_crop = rv[x0:x1, y0:y1, z0:z1]
    la_crop = la[x0:x1, y0:y1, z0:z1]
    ra_crop = ra[x0:x1, y0:y1, z0:z1]
    myo_crop = myo[x0:x1, y0:y1, z0:z1]
    
    case.cropped_shape = ct_crop.shape
    
    # Further trim empty slices
    # binary_mask = (sum of positive labels across all masks)  & turn the summed array back into a binary mask
    binary_mask = ((lv_crop + rv_crop + la_crop + ra_crop + myo_crop) > 0).astype(np.uint8)
    x_empty_indices = [i for i in range(binary_mask.shape[0]) if not np.any(binary_mask[i, :, :])]
    y_empty_indices = [i for i in range(binary_mask.shape[1]) if not np.any(binary_mask[:, i, :])]
    z_empty_indices = [i for i in range(binary_mask.shape[2]) if not np.any(binary_mask[:, :, i])]
    
    
    #print(f"cropped CT shape: {ct_crop.shape}, og_ct_voxel: {og_ct_voxel}")
    if x_empty_indices or y_empty_indices or z_empty_indices:
        case.log_error(
        f"Empty slices — "
        f"X: {x_empty_indices if x_empty_indices else 'None'}, "
        f"Y: {y_empty_indices if y_empty_indices else 'None'}, "
        f"Z: {z_empty_indices if z_empty_indices else 'None'} "
            )
    
    
    ct_res, ct_affine, ct_resampled = resample_CT(ct_crop, 
                                             og_ct_voxel, 
                                             original_affine=og_affine)
    lv_res, lv_affine, lv_resampled = resample_mask(lv_crop, ct_resampled, original_affine=og_affine)
    
    
def crop(case):
    # === Step 3 - Crop, trim, and resample heart ===
    lv = case.lv
    rv = case.rv
    la = case.la
    ra = case.ra
    myo = case.myo
    fullCT_data = case.ct_data
    fullCT_voxel = case.ct_voxel
    
    binary_mask = ((lv + rv + la + ra + myo) > 0).astype(np.uint8)   # (sum of positive labels across all masks)  & turn the summed array into a binary mask
    x_min, y_min, z_min = np.array(np.where(binary_mask)).min(axis=1)
    x_max, y_max, z_max = np.array(np.where(binary_mask)).max(axis=1)
    
    # calculate crop boundaries (3D bounding box) 
    # identify min and max voxel coordinates where heart structures are present
    x0, x1 = max(x_min, 0), min(x_max, og_ct_data.shape[0])
    y0, y1 = max(y_min, 0), min(y_max, og_ct_data.shape[1])
    z0, z1 = max(z_min, 0), min(z_max, og_ct_data.shape[2])
    
    case.log_error( f"3D bounding box coordinates- "
                    f"x: {x0}-{x1}, "
                    f"y: {y0}-{y1}, "
                    f"z: {z0}-{z1} ")
    
    # crop volumes with bounding box coordinates
    ct_crop = og_ct_data[x0:x1, y0:y1, z0:z1]
    lv_crop = lv[x0:x1, y0:y1, z0:z1]
    rv_crop = rv[x0:x1, y0:y1, z0:z1]
    la_crop = la[x0:x1, y0:y1, z0:z1]
    ra_crop = ra[x0:x1, y0:y1, z0:z1]
    myo_crop = myo[x0:x1, y0:y1, z0:z1]
    
    case.cropped_shape = ct_crop.shape
    
    # Further trim empty slices
    # binary_mask = (sum of positive labels across all masks)  & turn the summed array back into a binary mask
    binary_mask = ((lv_crop + rv_crop + la_crop + ra_crop + myo_crop) > 0).astype(np.uint8)
    x_empty_indices = [i for i in range(binary_mask.shape[0]) if not np.any(binary_mask[i, :, :])]
    y_empty_indices = [i for i in range(binary_mask.shape[1]) if not np.any(binary_mask[:, i, :])]
    z_empty_indices = [i for i in range(binary_mask.shape[2]) if not np.any(binary_mask[:, :, i])]
    
    
    #print(f"cropped CT shape: {ct_crop.shape}, og_ct_voxel: {og_ct_voxel}")
    if x_empty_indices or y_empty_indices or z_empty_indices:
        case.log_error(
        f"Empty slices — "
        f"X: {x_empty_indices if x_empty_indices else 'None'}, "
        f"Y: {y_empty_indices if y_empty_indices else 'None'}, "
        f"Z: {z_empty_indices if z_empty_indices else 'None'} "
            )
    

def append_case_to_csv(case_obj, csv_path):
    df = pd.DataFrame([case_obj.to_dict()])  
    write_header = not os.path.exists(csv_path)
    df.to_csv(csv_path, mode='a', index=False, header=write_header)        

CNTRL='CNTRL'
TTS='TTS'
        
def process_case(group):
    
    if group == NORMAL:
        CNTRL_dicom_root = f"../Takotsubo-Syndrome/data/Inputs/normal_cases" 
    elif group == TAKO:
        base_dicom_root = f"../Takotsubo-Syndrome/data/Inputs/takotsubo_cases"
    base_root = f"data/cases/"
    csv_path = f"data/case_summary.csv"
    
    # Step 1: Load already processed patient IDs from CSV
    #if os.path.exists(csv_path):
    #    existing_df = pd.read_csv(csv_path)
    #    already_processed = set(existing_df["Case"].astype(str))  # Assuming "Case" column stores patient_id
    #else:
    #    already_processed = set()

    for patientID in os.listdir(base_dicom_root):
        case_id=f"{group}_{patientID}"
        case = SampleInfo(patient_id=case_id, 
                          path=SamplePath(
              dicom_dir=os.path.join(base_dicom_root, patientID, "DICOM")
            , patient_dir=os.path.join(base_root, case_id)
            , segments_dir=os.path.join(base_root, case_id, "Segments")
            , cropped_dir=os.path.join(base_root, case_id, "Cropped")
            , resampled_dir=os.path.join(base_root, case_id, "Resampled")
            , ctSlices_dir=os.path.join(base_root, case_id, "CT_Slices")
            , segmentSlices_dir=os.path.join(base_root, case_id, "Segment_Slices")
            , pngSlices_dir=os.path.join(base_root, case_id, "PNG_Slices"))) 
        
        os.makedirs(case.path.patient_dir, exist_ok=True)
        os.makedirs(case.path.segments_dir, exist_ok=True)
        os.makedirs(case.path.cropped_dir, exist_ok=True)
        #os.makedirs(case.path.resampled_dir, exist_ok=True)
        #os.makedirs(case.path.ctSlices_dir, exist_ok=True)
        #os.makedirs(case.path.segmentSlices_dir, exist_ok=True)
        #os.makedirs(case.path.pngSlices_dir, exist_ok=True)


        
        #if case_id in already_processed:
        #    print(f"Skipping already processed: {case_id}")
        #    continue
        #print(f"Patient ID: {patientID}")
   
        
        
        # Load fullCT
        if not os.path.exists(case.path.fullCT):
            dicom2nifti.convert_dicom.dicom_series_to_nifti(case.path.dicom_dir, case.path.fullCT,)
            case.fullCTnib = nib.load(case.path.fullCT)
                
            _ = totalsegmentator(
                input_path=case.path.fullCT, 
                output_path=case.path.segments_dir,
                license_number="aca_BWYHC6UQQFDU8A",
                task="heartchambers_highres", 
                body_seg=True,
                preview=True,
                #ml=True,
                radiomics=True
            )
        
        
        crop(case)
        
        # Check for resampled image
        if not os.path.exists(case.path.croppedCT):
            case.log_error("cropped_ct.nii.gz not found")
            missing = [key for key, path in case.path.segments.items() if not os.path.exists(path)]
            case.missing_segments = missing
            append_case_to_csv(case, csv_path)
            continue

        # Process resampled image
        case.resampled = True
        resampled_img = nib.load(resampled_path)
        case.new_shape = resampled_img.shape
        case.new_voxel = tuple(round(s, 3) for s in resampled_img.header.get_zooms())
        case.new_orientation = aff2axcodes(resampled_img.affine)
        
        # Collect slice names
        for slice in os.listdir(nii_slice_dir):
            # if slice starts with ctX_ add file name to case.X_slices
            if slice.startswith("ctX_"):
                case.X_slices.append(slice)
            # if slice starts with ctY_ add file name to case.Y_slices
            elif slice.startswith("ctY_"):
                case.Y_slices.append(slice)
            # if slice starts with ctZ_ add file name to case.Z_slices
            elif slice.startswith("ctZ_"):
                case.Z_slices.append(slice)
            
        crop(case, segment_path)
        append_case_to_csv(case, csv_path)
        
        
        

In [6]:
process_case(TAKO)

Patient ID: AD 12191953
Patient ID: AG 11370442
Patient ID: AMH 36922300
Patient ID: AMS 22722383
Patient ID: APH 3516218
Patient ID: BB 8903260
Patient ID: BEH 4037842
Patient ID: BFM 10624765
Patient ID: BLT 11022191
Patient ID: CAMcD 10747764
Patient ID: CDA 38093134
Patient ID: CLB 50002879
Patient ID: CML 11113354
Patient ID: CMS 4664231
Patient ID: DCA 9939935
Patient ID: DD 26362236
Patient ID: DEM 9902248
Patient ID: DIW 10053411
Patient ID: DJD 6122410
Patient ID: DJK 2493
Patient ID: DJM 34687566
Patient ID: ECW 33674557
Patient ID: EIC 26844753
Patient ID: ELD 30527501
Patient ID: EMB 34404780
Patient ID: FFJ 11086170
Patient ID: GA 39800503
Patient ID: GAB 25512591
Patient ID: GB 10468072
Patient ID: GHM 4514352
Patient ID: GJB 30653190
Patient ID: GJI 25633926
Patient ID: HSM 3468857
Patient ID: IK 6526404
Patient ID: JAL 6996466
Patient ID: JAQ 3124096
Patient ID: JD 48020
Patient ID: JDMB 13788
Patient ID: JDMB 137882
Patient ID: JEB 6123921
Patient ID: JET 40340663
Pati

In [ ]:
case_path=f"{NORMAL}/SS 10937555_61F"   #OG shape (512, 86, 512)

Segment_root = f"data/Inputs/{case_path}"
new_output_root = f"data/New_Outputs/{case_path}"
os.makedirs(new_output_root, exist_ok=True)
crop_trim_resample_heart(Segment_root, new_output_root)

In [ ]:
'''
case=TAKO

patientID = f"{TAKO}/AG 11370442"
patientID = f"{TAKO}/JMB 38365177"
patientID = f"{TAKO}/JNMP 22916312"
patientID = f"{TAKO}/HSM 3468857"
patientID = f"{TAKO}/AMS 22722383"
patientID = f"{TAKO}/BFM 10624765"
patientID = f"{TAKO}/BLT 11022191"
patientID = f"{TAKO}/EIC 26844753"
patientID = f"{TAKO}/JDMB 137882"

OG_file_paths["LV"]
OG_file_paths["RV"]
OG_file_paths["LA"]
OG_file_paths["RA"]
OG_file_paths["MYO"]
OG_file_paths["CT"]

cropped_file_paths["Mask"]
cropped_file_paths["CT"]
cropped_file_paths["LV"]
cropped_file_paths["RV"]
cropped_file_paths["LA"]
cropped_file_paths["RA"]
cropped_file_paths["MYO"]

'''
import os
case=TAKO
patientID = "AG 11370442"

base_input_root = f"data/Inputs/{case}" #/{patient}
base_output_root = f"data/Outputs/{case}" #/{patient}
input_folder = os.path.join(base_input_root, patientID)
output_folder = os.path.join(base_output_root, patientID)
png_slices_folder=os.path.join(output_folder, "png_slices")

cropped_file_paths = get_cropped_file_paths(output_folder)
OG_file_paths = get_OG_file_paths(input_folder)

#slice_CT(output_folder)




In [ ]:
crop_trim_resample_heart

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import classification_report, roc_auc_score, roc_curve



class SingleBranchCNN(nn.Module):
    def __init__(self, in_channels=3):
        super(SingleBranchCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=3, padding=1)  # (B, 16, H, W)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2)                                        # (B, 16, H/2, W/2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)            # (B, 32, H/2, W/2)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2)                                        # (B, 32, H/4, W/4)
        
        self.dropout = nn.Dropout2d(0.3)  # Dropout for feature maps

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.dropout(x)
        return x  # Output shape: (B, 32, H/4, W/4)

class MultiViewCNN(nn.Module):
    def __init__(self, in_channels=3, input_size=(65, 65), use_metadata=False):
        super(MultiViewCNN, self).__init__()

        self.use_metadata = use_metadata
        H, W = input_size

        # Three branches
        self.axial_branch = SingleBranchCNN(in_channels)
        self.sagittal_branch = SingleBranchCNN(in_channels)
        self.coronal_branch = SingleBranchCNN(in_channels)

        # Output feature map size after 2x MaxPool2d(2)
        flattened_size = 32 * (H // 4) * (W // 4)  # each branch output flattened

        total_features = 3 * flattened_size       # all branches
        if use_metadata:
            total_features += 2  # e.g. age and gender

        # Fully connected classifier
        self.dropout1 = nn.Dropout(0.3)
        self.fc1 = nn.Linear(total_features, 128)
        self.dropout2 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 1)  # Binary classification (output: logit)

    def forward(self, axial, sagittal, coronal, meta=None):
        a = self.axial_branch(axial)
        s = self.sagittal_branch(sagittal)
        c = self.coronal_branch(coronal)

        a = a.view(a.size(0), -1)  # flatten
        s = s.view(s.size(0), -1)
        c = c.view(c.size(0), -1)

        x = torch.cat([a, s, c], dim=1)

        if self.use_metadata and meta is not None:
            x = torch.cat([x, meta], dim=1)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.dropout2(x)
        x = self.fc2(x)  # no sigmoid here (use BCEWithLogitsLoss)
        return x


In [ ]:



metadata, labels, slices = load_dataset()
dataset = SliceDataset(
    slices_dict=slices,          
    metadata_dict=metadata,      
    labels_dict=labels           
)

takotsubo_count = sum(1 for sample in dataset if sample["label"].item() == 1)
normal_count = sum(1 for sample in dataset if sample["label"].item() == 0)

print(f"Takotsubo: {takotsubo_count}, Normal: {normal_count}")

dataset_loader = DataLoaderModule(dataset, batch_size=1)

# Model setup
model = MultiViewCNN(in_channels=3, input_size=(65, 65), use_metadata=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
#class_weight = takotsubo_count / normal_count if normal_count > 0 else 1.0
pos_weight = torch.tensor([takotsubo_count / normal_count])  
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight.to(device))

#optimizer = optim.Adam(model.parameters(), lr=1e-4)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5, verbose=True)
train_loader = dataset_loader.get_train_loader()
val_loader = dataset_loader.get_val_loader()
test_loader = dataset_loader.get_test_loader()

# Early stopping parameters
best_val_loss = float('inf')
threshold_cutoff = 0.5  # threshold for binary classification
patience = 3
patience_counter = 0
num_epochs = 50  # longer max epochs now

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch in train_loader:
        x_axial = batch["axial"].to(device)
        x_sagittal = batch["sagittal"].to(device)
        x_coronal = batch["coronal"].to(device)
        x_meta = batch["meta"].to(device)
        y = batch["label"].to(device)

        outputs = model(x_axial, x_sagittal, x_coronal, x_meta)
        loss = criterion(outputs, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        predicted = (torch.sigmoid(outputs) > threshold_cutoff).float()
        total += y.size(0)
        correct += (predicted == y).sum().item()

    train_acc = 100 * correct / total
    avg_train_loss = running_loss / len(train_loader)

    # ====== Validation ======
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            x_axial = batch["axial"].to(device)
            x_sagittal = batch["sagittal"].to(device)
            x_coronal = batch["coronal"].to(device)
            x_meta = batch["meta"].to(device)
            y = batch["label"].to(device)

            outputs = model(x_axial, x_sagittal, x_coronal, x_meta)
            loss = criterion(outputs, y)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Train Acc: {train_acc:.2f}% | LR: {current_lr:.6f}")
    scheduler.step(avg_val_loss)
    # ====== Early stopping check ======
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pt")  # save best model
    else:
        patience_counter += 1
        print(f"No improvement in val loss ({patience_counter}/{patience})")

        if patience_counter >= patience:
            print("Early stopping triggered.")
            break



# ========== TEST EVALUATION WITH ROC AUC ========== #


# Load best model
torch.save(model.state_dict(), "best_model.pt")
model.load_state_dict(torch.load("best_model.pt"))
model.eval()

y_true = []
y_prob = []
threshold_cutoff = 0.5  # threshold for binary classification
with torch.no_grad():
    for batch in test_loader:
        x_axial = batch["axial"].to(device)
        x_sagittal = batch["sagittal"].to(device)
        x_coronal = batch["coronal"].to(device)
        x_meta = batch["meta"].to(device)
        y = batch["label"].to(device)

        outputs = model(x_axial, x_sagittal, x_coronal, x_meta)
        probs = torch.sigmoid(outputs).cpu().numpy()  # shape (B, 1)

        y_true.extend(y.cpu().numpy())
        y_prob.extend(probs)

# Flatten
y_true = [int(x) for x in y_true]
y_prob = [float(p) for p in y_prob]
y_pred = [int(p > threshold_cutoff) for p in y_prob]

# Report
print("\n📊 Classification Report:")
print(classification_report(y_true, y_pred, target_names=["Normal", "Takotsubo"]))

# ROC AUC
auc_score = roc_auc_score(y_true, y_prob)
print(f"🎯 ROC AUC Score: {auc_score:.4f}")

# ROC Curve plot (optional)
fpr, tpr, _ = roc_curve(y_true, y_prob)
plt.figure()
plt.plot(fpr, tpr, label=f"AUC = {auc_score:.2f}")
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.grid()
plt.show()

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Normal", "Takotsubo"])


plt.figure(figsize=(5, 4))
disp.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix")
plt.grid(False)
plt.show()
        

Processing normal_cases
Processing takotsubo_cases
Takotsubo: 81, Normal: 75


C:\Users\sulei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/50] | Train Loss: 0.9971 | Val Loss: 0.6446 | Train Acc: 52.29% | LR: 0.000100
Epoch [2/50] | Train Loss: 0.6598 | Val Loss: 0.5989 | Train Acc: 65.14% | LR: 0.000100
Epoch [3/50] | Train Loss: 0.5735 | Val Loss: 0.5771 | Train Acc: 67.89% | LR: 0.000100
Epoch [4/50] | Train Loss: 0.5403 | Val Loss: 0.5984 | Train Acc: 72.48% | LR: 0.000100
No improvement in val loss (1/3)
Epoch [5/50] | Train Loss: 0.4076 | Val Loss: 0.5515 | Train Acc: 82.57% | LR: 0.000100
Epoch [6/50] | Train Loss: 0.4270 | Val Loss: 0.5997 | Train Acc: 82.57% | LR: 0.000100
No improvement in val loss (1/3)
Epoch [7/50] | Train Loss: 0.3281 | Val Loss: 0.4957 | Train Acc: 87.16% | LR: 0.000100
Epoch [8/50] | Train Loss: 0.3458 | Val Loss: 0.5179 | Train Acc: 88.07% | LR: 0.000100
No improvement in val loss (1/3)
Epoch [9/50] | Train Loss: 0.2221 | Val Loss: 0.6800 | Train Acc: 89.91% | LR: 0.000100
No improvement in val loss (2/3)
Epoch [10/50] | Train Loss: 0.1738 | Val Loss: 0.5386 | Train Acc: 95.41% | 

In [ ]:
class MetadataModelWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, meta):
        # Create dummy tensors for image branches
        batch_size = meta.size(0)
        dummy = torch.zeros((batch_size, 3, 65, 65)).to(meta.device)
        return self.model(dummy, dummy, dummy, meta)
    
class MultiModalWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, axial, sagittal, coronal, meta):
        return self.model(axial, sagittal, coronal, meta)

In [ ]:
# Collect background (for DeepExplainer) and test examples
bg_axial, bg_sagittal, bg_coronal, bg_meta = [], [], [], []
test_axial, test_sagittal, test_coronal, test_meta = [], [], [], []

with torch.no_grad():
    for i, batch in enumerate(dataset_loader.get_val_loader()):
        if i >= 10: break  # Limit background
        bg_axial.append(batch["axial"])
        bg_sagittal.append(batch["sagittal"])
        bg_coronal.append(batch["coronal"])
        bg_meta.append(batch["meta"])

    for batch in dataset_loader.get_test_loader():
        test_axial.append(batch["axial"])
        test_sagittal.append(batch["sagittal"])
        test_coronal.append(batch["coronal"])
        test_meta.append(batch["meta"])

# Stack all background and test tensors
bg_input = [
    torch.cat(bg_axial).to(device),
    torch.cat(bg_sagittal).to(device),
    torch.cat(bg_coronal).to(device),
    torch.cat(bg_meta).to(device),
]

test_input = [
    torch.cat(test_axial).to(device),
    torch.cat(test_sagittal).to(device),
    torch.cat(test_coronal).to(device),
    torch.cat(test_meta).to(device),
]

#explainer = shap.DeepExplainer(MultiModalWrapper(model), bg_input)
#shap_values = explainer.shap_values(test_input, check_additivity=False)

explainer = shap.GradientExplainer(MultiModalWrapper(model), bg_input)
shap_values = explainer.shap_values(test_input)

meta_shap = shap_values[3]               # SHAP values for metadata branch (shape: [n_samples, 2])
meta_shap = np.squeeze(meta_shap)
meta_input = test_input[3].cpu().numpy() # Original input values for metadata

print("meta_shap shape:", meta_shap.shape)       # should be (n_samples, 2)
print("meta_input shape:", meta_input.shape)

shap.summary_plot(
    meta_shap,
    features=meta_input,
    feature_names=["Age", "Gender"]
)

shap_axial = shap_values[0][:, 0, :, :, 0]       # shape (24, 64, 64)
image_axial = test_input[0][:, 0, :, :].cpu().numpy()  # shape (24, 64, 64)
print(f"shap_axial shape: {shap_axial.shape}, image_axial shape: {image_axial.shape}")
shap_coronal = shap_values[0][:, 1, :, :, 0]
image_coronal = test_input[1][:, 0, :, :].cpu().numpy()
print(f"shap_coronal shape: {shap_coronal.shape}, image_coronal shape: {image_coronal.shape}")
shap_sagittal = shap_values[0][:, 2, :, :, 0]
image_sagittal = test_input[2][:, 0, :, :].cpu().numpy()
print(f"shap_sagittal shape: {shap_sagittal.shape}, image_sagittal shape: {image_sagittal.shape}")

titles = ["Axial", "Coronal", "Sagittal"]
for shap_img, input_img, title in zip(
    [shap_axial, shap_coronal, shap_sagittal],
    [image_axial, image_coronal, image_sagittal],
    titles):
    print(f"Plotting SHAP for {title} view")
    shap.image_plot(shap_img, input_img)

In [ ]:
''' 

# Load image data
ct_img = nib.load(cropped_file_paths["CT"])
mask_img = nib.load(cropped_file_paths["Mask"])
lv_img = nib.load(cropped_file_paths["LV"])
rv_img = nib.load(cropped_file_paths["RV"])
la_img = nib.load(cropped_file_paths["LA"])
ra_img = nib.load(cropped_file_paths["RA"])
myo_img = nib.load(cropped_file_paths["MYO"])
ct_cropped = ct_img.get_fdata()
mask_data = mask_img.get_fdata()
lv_data = lv_img.get_fdata()
rv_data = rv_img.get_fdata()
la_data = la_img.get_fdata()
ra_data = ra_img.get_fdata()
myo_data = myo_img.get_fdata()

'''

In [ ]:
plot_slices_from_png(png_slices_folder)
crop_trim_resample_heart(input_folder, output_folder)
plot_slices_masks(png_slices_folder, cropped_file_paths["CT"], cropped_file_paths["Mask"])

In [ ]:
'''
og_lv =  nib.load(OG_file_paths["LV"]).get_fdata()
og_rv =  nib.load(OG_file_paths["RV"]).get_fdata()
og_la =  nib.load(OG_file_paths["LA"]).get_fdata()
og_ra =  nib.load(OG_file_paths["RA"]).get_fdata()
og_myo = nib.load(OG_file_paths["MYO"]).get_fdata()
og_ct =  nib.load(OG_file_paths["CT"]).get_fdata()

ct_cropped = nib.load(cropped_file_paths["CT"]).get_fdata()
lv_cropped = nib.load(cropped_file_paths["LV"]).get_fdata()
rv_cropped = nib.load(cropped_file_paths["RV"]).get_fdata()
la_cropped = nib.load(cropped_file_paths["LA"]).get_fdata()
ra_cropped = nib.load(cropped_file_paths["RA"]).get_fdata()
myo_cropped = nib.load(cropped_file_paths["MYO"]).get_fdata()
mask = nib.load(cropped_file_paths["Mask"]).get_fdata()

'''

check_original_mask_alignment(ct_path=OG_file_paths["CT"],mask_path=OG_file_paths["LV"],world_mm=-125.8125,slice_axis='z',slice_index=170)

ct_img = nib.load(OG_file_paths["CT"])
ct_data = ct_img.get_fdata()
print("original Shape:", ct_img.shape)
print("original Orientation:", aff2axcodes(ct_img.affine))  

cropped_img = nib.load(cropped_file_paths["CT"])
cropped_data = cropped_img.get_fdata()
print("Cropped Shape:", cropped_img.shape)
print("Cropped Orientation:", aff2axcodes(cropped_img.affine))  

original Shape: (512, 512, 311)
original Orientation: ('L', 'A', 'S')
Cropped Shape: (195, 149, 138)
Cropped Orientation: ('R', 'A', 'S')


In [2]:
import pydicom
import os

tako='takotsubo_cases'
case=tako
patientID = "XZC 32444390"
base_dicom_root = f"../Takotsubo-Syndrome/data/Inputs/{case}/{patientID}/DICOM/Chest C+"  # {patient}/DICOM

for filename in os.listdir(base_dicom_root):
    if filename.endswith(".dcm"):
        dcm = pydicom.dcmread(os.path.join(base_dicom_root, filename))
        print(f"Patient: {dcm.PatientName}")
        print(f"Modality: {dcm.Modality}")
        print(f"Series Description: {dcm.SeriesDescription}")
        print(f"Slice Thickness: {dcm.SliceThickness} mm")
        print(f"Pixel Spacing: {dcm.PixelSpacing} mm")
        print(f"KVP (tube voltage): {dcm.get('KVP', 'N/A')} kV")
        print(f"Exposure Time: {dcm.get('ExposureTime', 'N/A')} ms")
        break  # Only inspect one slice for now
        

Patient: 1048355585
Modality: CT
Series Description: Chest C+
Slice Thickness: 1.25 mm
Pixel Spacing: [0.615234, 0.615234] mm
KVP (tube voltage): 120 kV
Exposure Time: 600 ms


In [3]:
import pydicom
import os
import csv

# Define base directory
tako = 'takotsubo_cases'
base_root = f"../Takotsubo-Syndrome/data/Inputs/{tako}"

# Define output CSV
output_csv = "dicom_metadata_summary.csv"
fieldnames = [
    "PatientFolder",
    "PatientName",
    "Modality",
    "SeriesDescription",
    "SliceThickness_mm",
    "PixelSpacing_mm",
    "KVP_kV",
    "ExposureTime_ms"
]

# Collect metadata
with open(output_csv, mode='w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for patient_folder in os.listdir(base_root):
        patient_path = os.path.join(base_root, patient_folder)
        if not os.path.isdir(patient_path):
            continue

        # Search for the first .dcm file recursively
        dicom_file_found = False
        for root, dirs, files in os.walk(patient_path):
            for file in files:
                if file.lower().endswith(".dcm"):
                    dicom_path = os.path.join(root, file)
                    try:
                        dcm = pydicom.dcmread(dicom_path, stop_before_pixels=True)
                        writer.writerow({
                            "PatientFolder": patient_folder,
                            "PatientName": getattr(dcm, "PatientName", "N/A"),
                            "Modality": getattr(dcm, "Modality", "N/A"),
                            "SeriesDescription": getattr(dcm, "SeriesDescription", "N/A"),
                            "SliceThickness_mm": getattr(dcm, "SliceThickness", "N/A"),
                            "PixelSpacing_mm": getattr(dcm, "PixelSpacing", "N/A"),
                            "KVP_kV": dcm.get("KVP", "N/A"),
                            "ExposureTime_ms": dcm.get("ExposureTime", "N/A"),
                        })
                        dicom_file_found = True
                        break  # Only inspect one DICOM file per patient
                    except Exception as e:
                        print(f"Failed to read {dicom_path}: {e}")
                        break
            if dicom_file_found:
                break
        break 

print(f"Metadata extracted to {output_csv}")

Metadata extracted to dicom_metadata_summary.csv
